In [112]:
'''
#Sequence to sequence example in Keras (character-level).

This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

**Summary of the algorithm**

- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.

**Data download**

[English to French sentence pairs.
](http://www.manythings.org/anki/fra-eng.zip)

[Lots of neat sentence pairs datasets.
](http://www.manythings.org/anki/)

**References**

- [Sequence to Sequence Learning with Neural Networks
   ](https://arxiv.org/abs/1409.3215)
- [Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    ](https://arxiv.org/abs/1406.1078)
'''

'\n#Sequence to sequence example in Keras (character-level).\n\nThis script demonstrates how to implement a basic character-level\nsequence-to-sequence model. We apply it to translating\nshort English sentences into short French sentences,\ncharacter-by-character. Note that it is fairly unusual to\ndo character-level machine translation, as word-level\nmodels are more common in this domain.\n\n**Summary of the algorithm**\n\n- We start with input sequences from a domain (e.g. English sentences)\n    and corresponding target sequences from another domain\n    (e.g. French sentences).\n- An encoder LSTM turns input sequences to 2 state vectors\n    (we keep the last LSTM state and discard the outputs).\n- A decoder LSTM is trained to turn the target sequences into\n    the same sequence but offset by one timestep in the future,\n    a training process called "teacher forcing" in this context.\n    It uses as initial state the state vectors from the encoder.\n    Effectively, the decoder 

In [113]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import EarlyStopping
import numpy as np

In [114]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.  REMY YOU STUPID SLUT Set to 600 
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'                                   # REMY YOU STUPID SLUT Set to English/Simple English

In [115]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [116]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# Construct empty tensors to store our data
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [117]:
# Implement one-hot encoding
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [118]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [119]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [120]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

early_stop = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5
)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
          callbacks=[early_stop],
          verbose=2)


# Save model
model.save('s2s_early_stop.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
 - 66s - loss: 0.9155 - val_loss: 0.9273
Epoch 2/100
 - 54s - loss: 0.7216 - val_loss: 0.7587
Epoch 3/100
 - 54s - loss: 0.6148 - val_loss: 0.6739
Epoch 4/100
 - 54s - loss: 0.5600 - val_loss: 0.6268
Epoch 5/100
 - 54s - loss: 0.5203 - val_loss: 0.5930
Epoch 6/100
 - 56s - loss: 0.4884 - val_loss: 0.5628
Epoch 7/100
 - 56s - loss: 0.4624 - val_loss: 0.5382
Epoch 8/100
 - 56s - loss: 0.4401 - val_loss: 0.5322
Epoch 9/100
 - 56s - loss: 0.4203 - val_loss: 0.5086
Epoch 10/100
 - 55s - loss: 0.4019 - val_loss: 0.5088
Epoch 11/100
 - 56s - loss: 0.3861 - val_loss: 0.4917
Epoch 12/100
 - 56s - loss: 0.3713 - val_loss: 0.4688
Epoch 13/100
 - 56s - loss: 0.3578 - val_loss: 0.4724
Epoch 14/100
 - 58s - loss: 0.3456 - val_loss: 0.4602
Epoch 15/100
 - 63s - loss: 0.3334 - val_loss: 0.4592
Epoch 16/100
 - 79s - loss: 0.3220 - val_loss: 0.4545
Epoch 17/100
 - 71s - loss: 0.3117 - val_loss: 0.4473
Epoch 18/100
 - 72s - loss: 0.3017 - val_lo

/Applications/anaconda3/lib/python3.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_6 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_5/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_5/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [121]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [122]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [127]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('Correct translation:', target_texts[seq_index])

-
Input sentence: Go.
Decoded sentence: Sourrez l'ispate !

Correct translation: 	Va !

-
Input sentence: Hi.
Decoded sentence: Aile-toi !

Correct translation: 	Salut !

-
Input sentence: Run!
Decoded sentence: Arrête de heure !

Correct translation: 	Cours !

-
Input sentence: Run!
Decoded sentence: Arrête de heure !

Correct translation: 	Courez !

-
Input sentence: Who?
Decoded sentence: Qui est le mon ?

Correct translation: 	Qui ?

-
Input sentence: Wow!
Decoded sentence: Attends juste !

Correct translation: 	Ça alors !

-
Input sentence: Fire!
Decoded sentence: Allez vous arrêter !

Correct translation: 	Au feu !

-
Input sentence: Help!
Decoded sentence: Salut !

Correct translation: 	À l'aide !

-
Input sentence: Jump.
Decoded sentence: Appolle !

Correct translation: 	Saute.

-
Input sentence: Stop!
Decoded sentence: Arrête de tiller.

Correct translation: 	Ça suffit !

-
Input sentence: Stop!
Decoded sentence: Arrête de tiller.

Correct translation: 	Stop !

-
Input sentenc

-
Input sentence: Be nice.
Decoded sentence: Soyez clémente !

Correct translation: 	Soyez gentilles !

-
Input sentence: Beat it.
Decoded sentence: Regarde ça !

Correct translation: 	Dégage !

-
Input sentence: Call me.
Decoded sentence: Appelle-moi !

Correct translation: 	Appelle-moi !

-
Input sentence: Call me.
Decoded sentence: Appelle-moi !

Correct translation: 	Appellez-moi !

-
Input sentence: Call us.
Decoded sentence: Appelle-moi !

Correct translation: 	Appelle-nous !

-
Input sentence: Call us.
Decoded sentence: Appelle-moi !

Correct translation: 	Appelez-nous !

-
Input sentence: Come in.
Decoded sentence: Viens ici !

Correct translation: 	Entrez !

-
Input sentence: Come in.
Decoded sentence: Viens ici !

Correct translation: 	Entre.

-
Input sentence: Come in.
Decoded sentence: Viens ici !

Correct translation: 	Entre !



In [41]:
your_string = "Be nice."

In [42]:
your_seq = np.zeros(
    (1, max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

In [43]:
for index,character in enumerate(your_string):
    your_seq[0, index, input_token_index[character]] = 1.

In [44]:
your_translation = decode_sequence(your_seq)
your_translation

'Soyez gentilles !\n'

In [90]:
idx = 16

In [91]:
print(input_texts[idx])
print(target_texts[idx])
print(decode_sequence(encoder_input_data[idx:idx+1]))

Go on.
	Poursuivez.

Poursuivez.



In [92]:
true = target_texts[idx].split('\t')[1]

In [93]:
pred = decode_sequence(encoder_input_data[idx:idx+1])

In [94]:
true

'Poursuivez.\n'

In [95]:
pred

'Poursuivez.\n'

In [96]:
from nltk.translate.bleu_score import sentence_bleu
sentence_bleu([[true]],[pred])

1.821831989445342e-231

In [100]:
from nltk.translate.bleu_score import sentence_bleu
reference = [['this', 'is', 'a', 'test'], ['this', 'is' 'test']]
candidate = ['this', 'is', 'a', 'test']
score = sentence_bleu(reference, candidate)
print(score)

1.0


In [84]:
# two references for one document
from nltk.translate.bleu_score import corpus_bleu
references = [[['this', 'is', 'a', 'test'], ['this', 'is' 'test']]]
candidates = [['this', 'is', 'a', 'test']]
score = corpus_bleu(references, candidates)
print(score)

1.0
